In [ ]:
import metatlas
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# myPath = '/global/homes/b/bpb/ExoMetabolomic_Example_Data/'
myPath = '/project/projectdirs/metatlas/original_data/tls/150330_bundle_timecourse/'
%system ls $myPath
# myFile = 'MEDIA-1.mzML'
myFile = '150405_2A_1day_neg.mzML'
metatlas.mzml_to_hdf('%s%s'%(myPath,myFile))

In [ ]:
import tables
fid = tables.open_file('%s%s'%(myPath,myFile.replace('mzML','h5')))

In [ ]:
polarity = 0
# mz_theor = 205.0971541 #tryptophan
# mz_theor = 150.0583 #methionine
mz_theor = 386.999943 #Leu-Leu
rt,intensity = metatlas.get_XIC(fid, 0, 1500, 1, polarity)
metatlas.plot_XIC(rt,intensity)

In [ ]:
polarity = 0
# mz_theor = 205.0971541 #tryptophan
# mz_theor = 150.0583 #methionine
mz_theor = 133.051136
rt,intensity = metatlas.get_XIC(fid, mz_theor - mz_theor*50/1e6, mz_theor + mz_theor*50/1e6, 1, polarity)

In [ ]:
%matplotlib inline
metatlas.plot_XIC(rt,intensity)
plt.xlim(3,7)

In [ ]:
# get coarse 2d hist
mzEdges = np.linspace(80, 620,500)
rtEdges = np.linspace(2,15,500)
ms_level = 1
polarity = 0
hMap = metatlas.get_heatmap(fid,mzEdges,rtEdges,ms_level,polarity)
metatlas.plot_heatmap(np.log10(hMap['arr']+1),hMap['rt_bins'],hMap['mz_bins'],title='entire file')
plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
# fig.savefig('test2png.png', dpi=100)

In [ ]:

ms_level=2
rt_min = 3.0
rt_max = 3.5
data = metatlas.get_data(fid, ms_level, polarity,
                    min_mz=0,
                    max_mz=mz_theor+10,
                    min_rt=rt_min,
                    max_rt=rt_max,
                    min_precursor_MZ=mz_theor -  mz_theor*10/1e6,
                    max_precursor_MZ=mz_theor +  mz_theor*10/1e6)
#                     min_precursor_intensity=0,
#                     max_precursor_intensity=0,
#                     min_collision_energy=0,
#                     max_collision_energy=0)
plt.vlines(data['mz'],0,data['i'],color='r',linestyles='solid')
plt.xlabel('m/z')
plt.ylabel('intensity')
# plt.xlim(0,225)
# plt.plot(data['mz'],data['i'])
plt.show()
idx = np.argsort(data['i'])
print data['mz'][idx]


In [ ]:
from metatlas import xcms
import os
myFileList = []
for file in os.listdir(myPath):
    if file.endswith(".mzML"):
        if 'neg' in file:
            myFileList.append('%s%s'%(myPath,file))
print myFileList
xset = xcms.get_xmcs_set(myFileList)
xset = xcms.group(xset)
df = xcms.peak_table(xset)
print(df.head())

In [ ]:
# print df['mz'][0]
# print df['STRAIN2.1'][0]
myKeys = np.sort(df.keys()[8:])
M = np.zeros((len(df),len(myKeys)))
for i,k in enumerate(myKeys):
    for j,d in enumerate(df[k]):
        M[j,i] = float(d)
M = np.nan_to_num(M)

In [ ]:
df

In [ ]:
m_idx = np.argmax(M,axis = 1)
m = np.max(M,axis = 1)
idx = np.argsort(m)[::-1]

In [ ]:
print df['mz'][idx[:5]]
# df['rt'][idx[:5]]
# missing = 170.15

In [ ]:
print np.sort(abs(df['rt'][idx[0]] - df['rt']))[:10]

In [ ]:
df.to_csv('saving_dataframe.xls')